In [63]:
!pip install numpy scipy networkx

Generating synthetic data

In [64]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.linalg import eigh, inv, det
import math

In [65]:
def generate_time_varying_precision(p=10, N=50, eps=1e-6, soft_lambda=0.14):
    """
    Generate a list of time-varying precision matrices Omega(t),
    following the four steps (i)-(iv) described:

      (i)  B_i are lower-triangular with entries ~ N(0,1/2).
      (ii) G(t) = ( sum_i B_i * phi_i(t) ) / 2.
      (iii) Omega^o(t) = G(t)G(t)^T, then soft-threshold off-diagonals.
      (iv) Add log10(p)/4 to the diagonal for positive definiteness.
    """

    # 1. Generate lower-triangular matrices B1,B2,B3,B4
    #    with elements from Normal(0, 1/2).
    B_list = [np.tril(np.random.normal(0, np.sqrt(0.5), (p, p))) 
              for _ in range(4)]

    # 2. Create time grid from 0 to 1
    t_values = np.linspace(0, 1, N)
    Omega_list = []

    for t in t_values:
        # Define phi_1(t), phi_2(t), phi_3(t), phi_4(t)
        phi = [
            np.sin(np.pi * t / 2),
            np.cos(np.pi * t / 2),
            np.sin(np.pi * t / 4),
            np.cos(np.pi * t / 4)
        ]

        # 3. Compute G(t) = (B1*phi1 + B2*phi2 + B3*phi3 + B4*phi4) / 2
        G = sum(B * ph for B, ph in zip(B_list, phi)) / 2

        # Omega^o(t) = G(t)*G(t)^T
        Omega_o = G @ G.T  # automatically symmetric

        # 4. Soft-threshold off-diagonal elements
        #    off_diag = sign(x) * max(|x|-lambda, 0)
        off_diag = Omega_o - np.diag(np.diag(Omega_o))  # zero out diag
        sign_off = np.sign(off_diag)
        mag_off = np.abs(off_diag)
        off_diag_thresh = sign_off * np.maximum(mag_off - soft_lambda, 0.0)

        # Put thresholded off-diagonals back, keep original diagonal
        Omega = np.diag(np.diag(Omega_o)) + off_diag_thresh

        # 5. Add log10(p)/4 to the diagonal to ensure positivity
        diag_adj = np.log10(p)/4 + eps
        Omega += np.eye(p) * diag_adj

        # 6. Double-check positive definiteness (just in case)
        eigvals, _ = eigh(Omega)
        if np.any(eigvals <= 0):
            # If not PD, shift up by |min_eig| + eps
            Omega += np.eye(p) * (abs(np.min(eigvals)) + eps)

        Omega_list.append(Omega)

    return Omega_list

def generate_synthetic_data(p=10, N=50):
    """
    Generate a time-series dataset X(t) by sampling from
    N(0, Sigma(t)), where Sigma(t) = Omega(t)^{-1}.
    The code returns:
      dataset: shape (N, p)
      Omega_list: list of precision matrices for each time grid
    """
    Omega_list = generate_time_varying_precision(p, N)

    dataset = np.zeros((N, p))
    for i, Omega in enumerate(Omega_list):
        # Invert each Omega(t) to get Sigma(t)
        try:
            Sigma = np.linalg.inv(Omega)
        except np.linalg.LinAlgError:
            Sigma = np.linalg.pinv(Omega)

        # Draw a single sample from N(0, Sigma)
        dataset[i] = multivariate_normal.rvs(
            mean=np.zeros(p),
            cov=Sigma,
            size=1
        )

    # Center the entire dataset
    dataset -= dataset.mean(axis=0)
    return dataset, Omega_list


Blockwise ADMM

In [66]:
def admm_loggle(
    S_dict,            # dict of empirical covariances: i -> \hat{Sigma}(t_i), each p x p
    time_indices,      # list or set of time indices i in Nk,d
    lam,               # penalty weight (omega in your text)
    rho,               # ADMM parameter (> 0), recommended ~ lam
    alpha=1.5,         # over-relaxation parameter
    abs_tol=1e-5,      # absolute tolerance for stopping
    rel_tol=1e-3,      # relative tolerance for stopping
    max_iter=1000,     # maximum ADMM iterations
    verbose=False
):
    """
    Solve the local group-lasso problem via ADMM for the set of time points in `time_indices`,
    i.e. Nk,d = { i : |t_i - t_k| <= d }.

    The objective is
       min_{Omega,Z} sum_{i in Nk,d} [ tr(Omega(t_i) * S(t_i)) - log det(Omega(t_i)) ]
                     + lam * sum_{u!=v} sqrt( sum_{i in Nk,d} Z_uv(t_i)^2 ),
    subject to Omega(t_i) - Z(t_i) = 0, Omega(t_i) >> 0, i in Nk,d.

    Returns dictionaries:
       Omega_dict, Z_dict, U_dict
    containing the final ADMM iterates for each time index.

    References:
    - The "loggle" step (i)-(iii) from your question.
    - Over-relaxation from Boyd et al. (2011) "Distributed Optimization and Statistical Learning via ADMM", Sec. 3.4.3.
    """

    # ---- 1) Initialization ----------------------------------
    # We'll store the ADMM iterates in dictionaries keyed by i in time_indices.
    # For convenience, let p be the dimension from the shape of any S(t_i).
    i0 = time_indices[0]
    p = S_dict[i0].shape[0]

    # Initialize Z^(0) and U^(0) to zero
    Z_dict = {i: np.zeros((p,p), dtype=float) for i in time_indices}
    U_dict = {i: np.zeros((p,p), dtype=float) for i in time_indices}
    Omega_dict = {i: np.zeros((p,p), dtype=float) for i in time_indices}  # will be updated

    # For faster updates, pre-compute identity
    I_p = np.eye(p)

    # ---- 2) ADMM loop ----------------------------------------
    for s in range(max_iter):

        # ---- (i) Update Omega^(s)(t_i) for each i in Nk,d ----
        for i in time_indices:
            # M = \hat{Sigma}(t_i) - rho ( Z^{s-1}(t_i) - U^{s-1}(t_i) )
            M = S_dict[i] - rho*(Z_dict[i] - U_dict[i])

            # Eigen-decompose M = Q * Lambda * Q^T
            # Then solve for Omega = Q * diag(omega_j) * Q^T
            # where omega_j = (-lambda_j + sqrt(lambda_j^2 + 4 rho)) / (2 rho).
            eigvals, Q = np.linalg.eigh(M)
            # Build the diagonal of solutions
            new_eigs = np.array([
                (-lmbd + np.sqrt(lmbd**2 + 4.0*rho)) / (2.0*rho)
                for lmbd in eigvals
            ])
            Omega_dict[i] = (Q * new_eigs) @ Q.T  # Q diag(new_eigs) Q^T

        # ---- Over-relaxation: O_bar = alpha*Omega + (1-alpha)*Z ----
        O_bar_dict = {}
        for i in time_indices:
            O_bar_dict[i] = alpha * Omega_dict[i] + (1.0 - alpha) * Z_dict[i]

        # ---- (ii) Update Z^(s)(t_i) for each i in Nk,d --------
        #
        #  - Diagonal: Z_{uu}(t_i) = O_bar_{uu}(t_i) + U_{uu}(t_i).
        #  - Off-diagonal: group-lasso across i in Nk,d for each (u,v).
        #    L_{uv} = sqrt( sum_{i} (O_bar_{uv}(t_i) + U_{uv}(t_i))^2 ).
        #    Then Z_{uv}(t_i) = max(1 - lam/(rho * L_{uv}), 0) * (O_bar_{uv}(t_i) + U_{uv}(t_i)).
        #
        # We do it in a vectorized way per (u,v). The group-lasso shrinks each (u,v) across all i simultaneously.
        Z_old = {i: Z_dict[i].copy() for i in time_indices}  # for dual residual

        # For each pair (u,v), gather the set of "O_bar_{uv} + U_{uv}" across i
        # We can do a double loop over u,v or a triple nested loop. 
        # For large p, you might want to vectorize carefully, but here we keep it more explicit.
        for u in range(p):
            for v in range(p):
                # Diagonal case: no shrink
                if u == v:
                    for i in time_indices:
                        Z_dict[i][u,v] = O_bar_dict[i][u,v] + U_dict[i][u,v]
                # Off-diagonal: group-lasso
                else:
                    # Collect the entire vector [ O_bar_{uv}(ti)+U_{uv}(ti) for i in Nk,d ]
                    big_vec = np.array([O_bar_dict[i][u,v] + U_dict[i][u,v] for i in time_indices])
                    norm_val = np.linalg.norm(big_vec, 2)  # L_{uv}

                    if norm_val == 0.0:
                        shrink = 0.0
                    else:
                        shrink = max(0.0, 1.0 - lam/(rho * norm_val))

                    # Update each Z_{uv}(t_i)
                    for idx_i, i in enumerate(time_indices):
                        Z_dict[i][u,v] = shrink * big_vec[idx_i]

        # ---- (iii) Update U^(s)(t_i) for each i in Nk,d -------
        for i in time_indices:
            # Standard ADMM dual update with *no* over-relaxation here
            U_dict[i] = U_dict[i] + (Omega_dict[i] - Z_dict[i])

        # ---- 3) Check stopping criteria (primal and dual residuals) ---
        # primal residual r^s = Omega^s - Z^s, dual residual d^s = Z^s - Z^(s-1)
        r_norm_sq = 0.0
        d_norm_sq = 0.0
        Om_norm_sq = 0.0
        Z_norm_sq = 0.0
        U_norm_sq = 0.0

        for i in time_indices:
            r_ij = Omega_dict[i] - Z_dict[i]
            d_ij = Z_dict[i] - Z_old[i]

            r_norm_sq += np.sum(r_ij**2)
            d_norm_sq += np.sum(d_ij**2)

            Om_norm_sq += np.sum(Omega_dict[i]**2)
            Z_norm_sq += np.sum(Z_dict[i]**2)
            U_norm_sq += np.sum(U_dict[i]**2)

        r_norm = np.sqrt(r_norm_sq)
        d_norm = np.sqrt(d_norm_sq)

        # primal feasibility tolerance
        eps_pri = abs_tol*np.sqrt(p*p*len(time_indices)) + \
                  rel_tol*max(np.sqrt(Om_norm_sq), np.sqrt(Z_norm_sq))

        # dual feasibility tolerance
        eps_dual = abs_tol*np.sqrt(p*p*len(time_indices)) + \
                   rel_tol*np.sqrt(U_norm_sq)

        if verbose and (s % 50 == 0):
            print(f"ADMM iter {s:4d}: r_norm={r_norm:.3e}, d_norm={d_norm:.3e},"
                  f" eps_pri={eps_pri:.3e}, eps_dual={eps_dual:.3e}")

        if (r_norm <= eps_pri) and (d_norm <= eps_dual):
            if verbose:
                print(f"Converged at iteration {s}")
            break

    return Omega_dict, Z_dict, U_dict

In [67]:
def gaussian_kernel(u, normalize=True):
    """
    Standard Gaussian kernel K(u) = exp(-u^2/2) / sqrt(2*pi), if normalize=True.
    If normalize=False, we drop 1/sqrt(2*pi) and just use exp(-u^2/2).
    """
    c = 1.0 / np.sqrt(2.0 * np.pi) if normalize else 1.0
    return c * np.exp(-0.5 * u**2)

In [68]:
def kernel_smoothed_cov(x_data, times, t_target, h, kernel_fun=gaussian_kernel):
    """
    Compute the kernel-smoothed covariance at a single target time t_target.

    Parameters
    ----------
    x_data : numpy array of shape (N, p)
        Observations x_j in R^p. The j-th row is x_j^T.
    times : numpy array of shape (N,)
        Observation times corresponding to each x_j.
    t_target : float
        The target time at which we want \hat{Sigma}(t_target).
    h : float
        Bandwidth for the kernel.
    kernel_fun : callable
        A kernel function K(u), e.g. Gaussian.

    Returns
    -------
    cov_matrix : (p, p) numpy array
        The kernel-smoothed covariance matrix at time t_target.
    """
    N, p = x_data.shape

    # Compute all weights w_j = K((t_j - t_target)/h).
    diffs = (times - t_target) / h
    weights = np.array([kernel_fun(d) for d in diffs])

    # Normalize weights so that they sum to 1.
    w_sum = weights.sum()
    if w_sum <= 1e-14:
        # Edge case: if no points get any weight, return something safe (e.g. zero or unweighted).
        # Or you could just return the empirical covariance unweighted.
        return np.cov(x_data.T, bias=True)

    w_normalized = weights / w_sum

    # Accumulate weighted outer products x_j x_j^T
    cov_matrix = np.zeros((p,p), dtype=float)
    for j in range(N):
        x_j = x_data[j]  # shape (p,)
        cov_matrix += w_normalized[j] * np.outer(x_j, x_j)

    return cov_matrix

In [69]:
def kernel_smoothed_covs_at_observed_times(x_data, times, h, kernel_fun=gaussian_kernel):
    """
    Compute kernel-smoothed covariance at each observed time in 'times'.

    Returns
    -------
    cov_dict : dict
        Keys: i = 0, 1, ..., N-1
        Values: (p,p) numpy arrays, \hat{Sigma}(t_i)
    """
    N = len(times)
    cov_dict = {}
    for i, t_i in enumerate(times):
        cov_dict[i] = kernel_smoothed_cov(x_data, times, t_i, h, kernel_fun)
    return cov_dict

In [70]:
def get_edge_set(Omega, threshold=1e-4):
    """
    Return the set of edges (u,v) with u < v where |Omega[u,v]| > threshold.
    Diagonal entries are excluded by construction.
    """
    p = Omega.shape[0]
    edges = set()
    for u in range(p):
        for v in range(u+1, p):
            if abs(Omega[u, v]) > threshold:
                edges.add((u, v))
    return edges

In [71]:
def compute_fdr_power_f1(true_edge_sets, est_edge_sets):
    """
    Inputs:
      true_edge_sets: list of sets S_k
      est_edge_sets : list of sets S^_k
    Returns: (FDR, power, F1)
    """
    K = len(true_edge_sets)
    if K == 0:
        return 0.0, 0.0, 0.0

    sum_tp_over_est = 0.0  # sum of (true positives / #est_edges) across k
    sum_tp_over_true = 0.0 # sum of (true positives / #true_edges) across k

    for k in range(K):
        S_k = true_edge_sets[k]
        Shat_k = est_edge_sets[k]
        if len(Shat_k) == 0:
            # If no edges estimated, fraction is 0
            tp_over_est = 0
        else:
            tp_over_est = len(S_k.intersection(Shat_k)) / len(Shat_k)

        if len(S_k) == 0:
            tp_over_true = 0
        else:
            tp_over_true = len(S_k.intersection(Shat_k)) / len(S_k)

        sum_tp_over_est += tp_over_est
        sum_tp_over_true += tp_over_true

    avg_tp_over_est = sum_tp_over_est / K  # (1 - FDR)
    avg_tp_over_true = sum_tp_over_true / K # power

    FDR = 1 - avg_tp_over_est
    power = avg_tp_over_true
    # F1 = 2 * (1-FDR)*power / ((1-FDR)+power)
    denom = (1 - FDR) + power
    if denom < 1e-15:
        F1 = 0.0
    else:
        F1 = 2.0 * (1 - FDR) * power / denom

    return FDR, power, F1

In [72]:
def compute_kl_divergence(true_prec_list, est_prec_list):
    """
    Inputs:
      true_prec_list: length-K list of "true" precision matrices
      est_prec_list : length-K list of "estimated" precision matrices
    Returns:
      scalar float for average KL divergence
    """
    K = len(true_prec_list)
    if K == 0:
        return 0.0

    p = true_prec_list[0].shape[0]
    kl_vals = []
    for k in range(K):
        O_true = true_prec_list[k]
        O_est  = est_prec_list[k]

        O_true_inv = inv(O_true)  # or store if you already have
        prod = O_est @ O_true_inv

        # KL part: tr( O_est * O_true^-1 ) - log det( O_est*O_true^-1 ) - p
        tr_part = np.trace(prod)
        # numeric safeguard in case det is near 0
        logdet_part = math.log(np.linalg.det(prod) + 1e-15)
        kl_val = tr_part - logdet_part - p
        kl_vals.append(kl_val)

    return np.mean(kl_vals)

Evaluation

In [73]:
p = 10
N = 50
dataset, Omega_list_true = generate_synthetic_data(p=p, N=N)
#  --> dataset: shape (N, p)
#  --> Omega_list_true: length-N list of "true" Omega(t_i)

# For convenience, let's define time points same as generate_time_varying_precision:
times = np.linspace(0, 1, N)

#  We'll get a "local" estimate at each i in 0..N-1 by running ADMM
#            on the neighborhood Nk,d = { j : |times[j]-times[i]| <= d }.
d = 0.1 # Example neighborhood half-width
lam = 0.1
rho = 0.1

# We'll store the final ADMM-estimated Omega(t_i) in a list
Omega_list_est = [None]*N

# First, compute kernel-smoothed covariance at each time for the entire dataset
# so that we have \hat{Sigma}(t_j) for j in 0..N-1
h = 0.08
S_dict_global = kernel_smoothed_covs_at_observed_times(dataset, times, h)

# Now, for each i, build the sub-dict S_dict_i from the relevant Nk,d
for i in range(N):
    # Find the local neighborhood
    Nk_d = [ j for j in range(N) if abs(times[j] - times[i]) <= d ]

    # Build the sub-dict S_dict for that neighborhood
    S_dict_local = { j: S_dict_global[j] for j in Nk_d }

    # Solve the local problem
    Omega_dict_i, Z_dict_i, U_dict_i = admm_loggle(
        S_dict=S_dict_local,
        time_indices=Nk_d,
        lam=lam,
        rho=rho,
        alpha=1.5,
        abs_tol=1e-5,
        rel_tol=1e-3,
        max_iter=300,
        verbose=False
    )

    # After ADMM converges, we want the estimate of Omega(t_i).
    # It's stored in Omega_dict_i[i], because i is in Nk_d.
    # (If i is not in Nk_d for some reason, we can't get it, but typically
    #  i is definitely in its own neighborhood.)
    Omega_list_est[i] = Omega_dict_i[i]

# -- Step 3: Evaluate performance across all i in 0..N-1
true_edge_sets = []
est_edge_sets = []
for i in range(N):
    # Extract edges from the "true" Omega and from the "estimated" one
    true_edges = get_edge_set(Omega_list_true[i], threshold=1e-3)
    est_edges  = get_edge_set(Omega_list_est[i], threshold=1e-2)

    true_edge_sets.append(true_edges)
    est_edge_sets.append(est_edges)

# Compute FDR, power, F1
FDR, power, F1 = compute_fdr_power_f1(true_edge_sets, est_edge_sets)

# Compute KL
KL = compute_kl_divergence(Omega_list_true, Omega_list_est)

# Print results
print("========== Evaluation Results ==========")
print(f"FDR   = {FDR:.4f}")
print(f"Power = {power:.4f}")
print(f"F1    = {F1:.4f}")
print(f"KL    = {KL:.4f}")


========== Evaluation Results ==========
FDR   = 0.3386
Power = 0.8766
F1    = 0.7539
KL    = 6.7728
